In [15]:
from lasagne.layers import *  
from lasagne.objectives import *
from lasagne.nonlinearities import *
import numpy as np
import theano

In [16]:
#FCN used softmax w/ xent, but they also tried sigmoid w/ xent and 
# did pretty well
def downsample_mask(mask_t, num_decimation_layers):
    '''gt_mask (Theano Tensor4) -> 768 by 1162 by num_classes
    num_decimation_layer (int) -> number of stride 2 or max pool layers'''
    net = InputLayer(input_var=mask_t, shape=(None,1,768,1152))
    for i in range(num_decimation_layers):
        net = MaxPool2DLayer(net,pool_size=5,stride=2)
    return get_output(net)
            

def get_classwise_scores(last_enc_layer, num_classes):
    '''last_enc_layer  (Theano Tensor4) -> last encoder layer
    num_classes (int) -> should be equal to depth of gt_mask'''
    
    #get hyperplanes for each class
    net = Conv2DLayer(last_enc_layer,
                      num_filters=num_classes,
                      filter_size=1,
                      nonlinearity=sigmoid)
    return get_output(net)
    
    
def calc_seg_loss(gt_coarse_mask, class_scores):
    '''gt_coarse_mask (Theano Tensor4) -> 5 by 11 by num_classes or something'''
    '''class_scores (Theano Tensor4) -> same as above'''
    loss = binary_crossentropy(class_scores, gt_coarse_mask).mean()
    return loss

In [5]:
def make_scores(num=10):
    return np.random.random((num,1,5,11)).astype('float32')
    

In [6]:
def make_mask(num=10):
    mask = np.random.binomial(1, 0.1, (num, 1, 768, 1152))
    return mask.astype('float32')

In [7]:
mask = make_mask()

In [8]:
mask_t = theano.tensor.tensor4('m')

In [9]:
x = theano.tensor.tensor4('x')

In [10]:
net = InputLayer(input_var=x, shape=(10,10, 18, 30))
net = Conv2DLayer(net, num_filters=1200, stride=2, filter_size=3, pad=1)

In [11]:
class_scores = get_classwise_scores(net, 1)

In [12]:
new_mask = downsample_mask(mask_t, 6)

In [13]:
loss = calc_seg_loss(new_mask, class_scores)

In [14]:
loss.eval({mask_t: mask, x:np.random.random(((10, 10, 18, 30))).astype('float32')})

array(0.6853165254247334)